# Resonator and Qubit Spectroscopy with HDAWG and UHFQA

This notebook demonstrates pulsed resonator and pulsed qubit spectroscopy experiments with the HDAWG and UHFQA.
In contrast to the SHF instruments, which can sweep their oscillator frequencies in real time (see [this notebook](https://github.com/zhinst/laboneq/blob/main/examples/basic_experiments.ipynb)), HDAWG and UHFQA require sweeps in near time.

In [ ]:
# convenience import for all LabOne Q software functionality
from laboneq.simple import *

# helper import
from laboneq.contrib.example_helpers.plotting.plot_helpers import (
    plot_results,
    plot_simulation,
)
from laboneq.contrib.example_helpers.example_notebook_helper import create_device_setup


In [ ]:
# create device setup
device_setup = create_device_setup(generation=1)
use_emulation = True


# 1. Pulsed Resonator Spectroscopy

Find the resonance frequency of the qubit readout resonator by looking at the transmission or reflection of a probe signal applied through the readout line.

## 1.1 Define the Experiment

In [ ]:
# define sweep parameter - sweep over frequency of readout pulse
start = -300e6
stop = 300e6
count = 21

frequency_sweep_parameter = LinearSweepParameter(
    uid="frequency_sweep", start=start, stop=stop, count=count
)

# define number of averages
average_exponent = 4  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment - uses only a readout pulse and a data acquisition line
exp = Experiment(
    uid="Resonator Spectroscopy",
    signals=[
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)

## experimental pulse sequence
# Define an acquisition loop of type SPECTROSCOPY
with exp.sweep(uid="sweep", parameter=frequency_sweep_parameter):
    with exp.acquire_loop_rt(
        uid="shots",
        count=pow(2, average_exponent),
        averaging_mode=AveragingMode.SEQUENTIAL,
        acquisition_type=AcquisitionType.SPECTROSCOPY,
    ):
        # readout pulse and data acquisition
        with exp.section(uid="spectroscopy"):
            exp.play(
                signal="measure", pulse=pulse_library.const(length=1e-6, amplitude=1.0)
            )
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                length=1e-6,
            )
        # relax time after readout - for signal processing and qubit relaxation to ground state
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)


In [ ]:
# calibration for qubit 0
calib_q0 = Calibration()
calib_q0["measure"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=frequency_sweep_parameter,
        modulation_type=ModulationType.HARDWARE,
    )
)


In [ ]:
# signal map for qubit 0
map_q0 = {
    "measure": "/logical_signal_groups/q0/measure_line",
    "acquire": "/logical_signal_groups/q0/acquire_line",
}


## 1.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

# run experiment
my_results = session.run(exp)

# plot measurement results
plot_results(my_results)


In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
show_pulse_sheet("Resonator Spectroscopy", session.compiled_experiment)


# 2. Pulsed Qubit Spectroscopy

Find the resonance frequency of the qubit by looking at the change in resonator transmission when sweeping the frequency of a qubit excitation pulse.

In [ ]:
## define pulses

# qubit drive pulse
const_iq_100ns = pulse_library.const(uid="const_iq_100ns", length=100e-9, amplitude=1.0)
# readout drive pulse
readout_pulse = pulse_library.const(uid="readout_pulse", length=400e-9, amplitude=1.0)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)


In [ ]:
# define sweep parameter - sweep over the frequency of a qubit excitation pulse
start = 40e6
stop = 200e6
count = 21

drive_frequency_sweep = LinearSweepParameter(
    uid="qubit_freq", start=start, stop=stop, count=count
)

# define number of averages
average_exponent = 4  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment - no explicit mapping to qubit lines
exp = Experiment(
    uid="Qubit Spectroscopy",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
with exp.sweep(uid="sweep", parameter=drive_frequency_sweep):
    with exp.acquire_loop_rt(
        uid="shots",
        count=pow(2, average_exponent),
        averaging_mode=AveragingMode.SEQUENTIAL,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        # qubit excitation pulse - frequency will be swept
        with exp.section(uid="qubit_excitation", alignment=SectionAlignment.RIGHT):
            exp.play(signal="drive", pulse=const_iq_100ns)
        # readout and data acquisition
        with exp.section(uid="qubit_readout", play_after="qubit_excitation"):
            # play readout pulse
            exp.play(signal="measure", pulse=readout_pulse)
            # signal data acquisition
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                kernel=readout_weighting_function,
            )
        # relax time after readout - for signal processing and qubit relaxation to ground state
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)


In [ ]:
# define experiment calibration - sweep over qubit drive frequency
exp_calib = Calibration()
exp_calib["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=drive_frequency_sweep,
        modulation_type=ModulationType.HARDWARE,
    )
)

# define signal maps for qubit 0
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}


## 2.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_calibration(exp_calib)
exp.set_signal_map(map_q0)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0
my_results = session.run(exp)

# plot measurement results
plot_results(my_results, phase=True)


In [ ]:
# Plot simulated output signals
plot_simulation(session.compiled_experiment, 0, 10e-6)
